In [12]:
import pandas as pd
import numpy as np
import os 
import time

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

def data_generate(data_type_filled):
    time_in = 12
    time_out = 6
    num_sample = data_type_filled['PM2.5'].shape[0] -time_in -time_out
    num_loc = data_type_filled['PM2.5'].shape[1] -3
    num_feature = len(data_type_filled)
    data = []
    label = []
    data_filled_array = {}
    for type_now in data_type_filled:
        data_filled_array[type_now] = data_type_filled[type_now].iloc[:,3:].values
        
    start = time.time()
    for i in range(0,num_sample):
        data_now = np.zeros([time_in, num_feature, num_loc])
        label_now = np.zeros([time_out,num_loc])
        feature_now = -1
        for type_now in data_type_filled:
            feature_now = feature_now +1
            if data_type_filled[type_now].iloc[i+time_in+time_out-1,0] < 20170708 :
                data_now[:,feature_now,:] = data_filled_array[type_now][i:i+time_in,:]
                if type_now == 'PM2.5':
                    label_now = data_filled_array[type_now][i+time_in:i+time_in+time_out,:]
            elif data_type_filled[type_now].iloc[i,0] > 20170708 :
                data_now[:,feature_now,:] = data_filled_array[type_now][i:i+time_in,:]
                if type_now == 'PM2.5':
                    label_now = data_filled_array[type_now][i+time_in:i+time_in+time_out,:]
        
        data_now = data_now.reshape((time_in*num_feature,num_loc))
        data.append(data_now)
        label.append(label_now)
        
    end = time.time()
#     print('data generating lasted: '+str(end-start))
                
    data = np.array(data)
    label = np.array(label)
    return data, label

def Evaluation(label, predict):
    MAE = np.mean(np.abs(label - predict))
    RMSE = np.power(np.mean(np.power(label - predict,2)) ,0.5)

    label_grade = label
    label_grade[label_grade < 35] = 1
    label_grade[label_grade > 250] = 6
    label_grade[label_grade > 150] = 5
    label_grade[label_grade > 115] = 4
    label_grade[label_grade > 75] = 3
    label_grade[label_grade > 35] = 2
    
    predict_grade = predict
    predict_grade[predict_grade < 35] = 1
    predict_grade[predict_grade > 250] = 6
    predict_grade[predict_grade > 150] = 5
    predict_grade[predict_grade > 115] = 4
    predict_grade[predict_grade > 75] = 3
    predict_grade[predict_grade > 35] = 2
    
    res = np.zeros(label_grade.shape)
    res[label_grade == predict_grade] = 1
    num_cor = res.sum()
    num_all = res.shape[0] * res.shape[1]
    prec = num_cor/num_all
    return MAE,RMSE,prec

In [2]:
# 读取数据
# 注: 20141231读取不了, 删了; 20151230,1231是空文件, 删了

# date_list = []
# data_folder = os.walk(r"data")  
# for path,dir_list,file_list in data_folder:  
#     for file_name in file_list:  
#         path_now = os.path.join(path, file_name)
#         if file_name[:13] == "beijing_extra" :
#             date_list.append(path_now)

# date_list.sort()
# data_extra = pd.read_csv(date_list[0])
# for i in range(1,len(date_list)):
#     print(date_list[i])
#     data_now = pd.read_csv(date_list[i])
#     data_extra = pd.concat([data_extra, data_now], axis=0)

# data_extra.to_csv("data_extra.csv")

In [3]:
data_all = pd.read_csv("data_all.csv")
# data_extra = pd.read_csv("data_extra.csv")

/data2/syh/ENTER/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# 删去无用的前三列
data_all = data_all.iloc[:,3:]
# data_extra = data_extra.iloc[:,3:]

# 删去缺失了一半PM2.5数据的植物园
data_all = data_all.drop(columns=['植物园'])

# 删去20170702-20170708
data_all = data_all.drop(index= range(148393,148456))

# 选取PM2.5数据
data_all = data_all[data_all['type']=='PM2.5']

In [5]:
data_all

,date,hour,type,万寿西宫,万柳,东四,东四环,东高村,丰台花园,云岗,...,房山,昌平,榆垡,永乐店,永定门内,琉璃河,西直门北,通州,门头沟,顺义
0,20140101.0,0.0,PM2.5,66.0,57.0,35.0,45.0,21.0,80.0,67.0,...,129.0,34.0,90.0,110.0,66.0,136.0,35.0,62.0,NaN,24.0
5,20140101.0,1.0,PM2.5,72.0,68.0,66.0,46.0,14.0,83.0,77.0,...,130.0,26.0,122.0,116.0,68.0,181.0,56.0,79.0,NaN,22.0
10,20140101.0,2.0,PM2.5,80.0,81.0,70.0,49.0,23.0,94.0,50.0,...,124.0,26.0,135.0,133.0,76.0,235.0,59.0,90.0,NaN,21.0
15,20140101.0,3.0,PM2.5,89.0,95.0,71.0,57.0,34.0,106.0,40.0,...,96.0,26.0,146.0,159.0,89.0,224.0,67.0,100.0,NaN,18.0
20,20140101.0,4.0,PM2.5,92.0,95.0,82.0,86.0,51.0,111.0,39.0,...,64.0,28.0,145.0,138.0,102.0,172.0,61.0,104.0,NaN,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270651,20200509.0,19.0,PM2.5,34.0,33.0,37.0,26.0,29.0,34.0,17.0,...,41.0,24.0,28.0,27.0,31.0,35.0,28.0,32.0,38.0,23.0
270656,20200509.0,20.0,PM2.5,40.0,37.0,38.0,32.0,28.0,45.0,18.0,...,47.0,27.0,32.0,36.0,35.0,48.0,44.0,37.0,45.0,26.0
270661,20200509.0,21.0,PM2.5,46.0,40.0,42.0,40.0,31.0,48.0,19.0,...,44.0,31.0,34.0,44.0,37.0,44.0,49.0,43.0,46.0,29.0
270666,20200509.0,22.0,PM2.5,47.0,42.0,45.0,44.0,42.0,44.0,18.0,...,48.0,32.0,30.0,49.0,34.0,38.0,54.0,47.0,40.0,31.0


In [6]:
# 查看全体数据缺失情况
na_count = data_all.isnull().sum().sort_values(ascending=False)
na_rate = na_count / len(data_all)
na_data = pd.concat([na_count,na_rate],axis=1,keys=['count','ratio'])
na_data

,count,ratio
南三环,4894,0.090422
琉璃河,4727,0.087336
东四环,4265,0.078801
八达岭,4038,0.074606
榆垡,3692,0.068214
通州,3085,0.056999
前门,2988,0.055207
东高村,2701,0.049904
密云水库,2572,0.047521
西直门北,2474,0.045710


In [7]:
# 划分数据集
data_train = data_all[data_all['date'] < 20200000]
data_train = data_train[data_train['date'] > 20150000]
data_val = data_all[data_all['date'] > 20200000]
data_test = data_all[data_all['date'] < 20150000]

In [8]:
# data_all_type = {}
# for data_type in data_all['type'][0:5]:
#     data_all_type[data_type] = data_all[data_all['type']==data_type]
# for data_type in data_extra['type'][0:7]:
#     data_all_type[data_type] = data_extra[data_extra['type']==data_type]

data_train_type = {}
for data_type in data_train['type'][0:5]:
    data_train_type[data_type] = data_train[data_train['type']==data_type]
data_val_type = {}
for data_type in data_val['type'][0:5]:
    data_val_type[data_type] = data_val[data_val['type']==data_type]
data_test_type = {}
for data_type in data_test['type'][0:5]:
    data_test_type[data_type] = data_test[data_test['type']==data_type]

In [9]:
# for type_now in data_all_type:
#     data_all_type[type_now] = data_all_type[type_now].fillna(method='ffill')
#     data_all_type[type_now] = data_all_type[type_now].fillna(method='bfill')
for type_now in data_train_type:
    data_train_type[type_now] = data_train_type[type_now].fillna(method='ffill')
    data_train_type[type_now] = data_train_type[type_now].fillna(method='bfill')
for type_now in data_val_type:
    data_val_type[type_now] = data_val_type[type_now].fillna(method='ffill')
    data_val_type[type_now] = data_val_type[type_now].fillna(method='bfill')
for type_now in data_test_type:
    data_test_type[type_now] = data_test_type[type_now].fillna(method='ffill')
    data_test_type[type_now] = data_test_type[type_now].fillna(method='bfill')

In [10]:
# 查看数据缺失情况
print('train')
for type_now in data_train_type:
    df = data_train_type[type_now] 
    na_count = df.isnull().sum().sort_values(ascending=False)
    na_rate = na_count / len(df)
    na_data = pd.concat([na_count,na_rate],axis=1,keys=['count','ratio'])
    print(na_data['count'].max())
print('val')
for type_now in data_val_type:
    df = data_val_type[type_now] 
    na_count = df.isnull().sum().sort_values(ascending=False)
    na_rate = na_count / len(df)
    na_data = pd.concat([na_count,na_rate],axis=1,keys=['count','ratio'])
    print(na_data['count'].max())
print('test')
for type_now in data_test_type:
    df = data_test_type[type_now] 
    na_count = df.isnull().sum().sort_values(ascending=False)
    na_rate = na_count / len(df)
    na_data = pd.concat([na_count,na_rate],axis=1,keys=['count','ratio'])
    print(na_data['count'].max())

train
0
val
0
test
0


In [13]:
# 制作时序数据集
train_data,train_label = data_generate(data_train_type)
val_data,val_label = data_generate(data_val_type)
test_data,test_label = data_generate(data_test_type)

In [14]:
# 训练并测试模型
print('training')
# models = [DecisionTreeRegressor(),RandomForestRegressor(),AdaBoostRegressor(),GradientBoostingRegressor()]
models = [DecisionTreeRegressor()]
train_predict_all = []
val_predict_all = []
test_predict_all = []

for model_now in models:
    print(model_now)
    start = time.time()
    train_predict = np.zeros(train_label.shape)
    val_predict = np.zeros(val_label.shape)
    test_predict = np.zeros(test_label.shape)
    
    for ti in range(train_label.shape[1]):
        for lo in range(train_label.shape[2]):
            model_now.fit(train_data[:,:,lo], train_label[:,ti,lo])
            train_predict[:,ti,lo] = model_now.predict(train_data[:,:,lo])
            val_predict[:,ti,lo] = model_now.predict(val_data[:,:,lo])
            test_predict[:,ti,lo] = model_now.predict(test_data[:,:,lo])
            
    train_predict_all.append(train_predict)
    val_predict_all.append(val_predict)
    test_predict_all.append(test_predict)
    end = time.time()
    print('training lasted: '+str(end-start))

training
DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')
training lasted: 138.5560667514801


In [15]:
# 评价模型性能

for m in range(len(models)):
    print(models[m])
    for i in range(6):
        MAE, RMSE,PREC= Evaluation(train_label[:,i,:], train_predict_all[m][:,i,:])
        print('time:'+str(i+1)+' '+'MAE = '+str(MAE)+' '+'RMSE = '+str(RMSE)+' '+'PREC = '+str(PREC))

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')
time:1 MAE = 0.05755627911112795 RMSE = 4.5687951204088835 PREC = 0.9992380694255651
time:2 MAE = 0.10265248162650273 RMSE = 6.248558319971609 PREC = 0.9988529556960187
time:3 MAE = 0.13969499127360008 RMSE = 7.406093289490354 PREC = 0.9983987012609882
time:4 MAE = 0.16772662046783168 RMSE = 8.247523388964364 PREC = 0.9980917165286388
time:5 MAE = 0.1899744400168173 RMSE = 8.883400909853973 PREC = 0.9978656264217057
time:6 MAE = 0.20929430989275408 RMSE = 9.361330408348476 PREC = 0.9974438681182527


In [16]:
for m in range(len(models)):
    print(models[m])
    for i in range(6):
        MAE, RMSE,PREC= Evaluation(val_label[:,i,:], val_predict_all[m][:,i,:])
        print('time:'+str(i+1)+' '+'MAE = '+str(MAE)+' '+'RMSE = '+str(RMSE)+' '+'PREC = '+str(PREC))

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')
time:1 MAE = 9.679572804253965 RMSE = 18.165833883267677 PREC = 0.8029595769819288
time:2 MAE = 15.319705239374976 RMSE = 28.131251669935562 PREC = 0.7135718297855856
time:3 MAE = 19.551062843156352 RMSE = 34.6657698619526 PREC = 0.6517073737628077
time:4 MAE = 23.04489643167282 RMSE = 39.42744093351716 PREC = 0.6055608282166999
time:5 MAE = 26.320268108095792 RMSE = 44.21323887161431 PREC = 0.5703383757190726
time:6 MAE = 29.04897550990073 RMSE = 48.877915607079686 PREC = 0.5420403261732747


In [17]:
for m in range(len(models)):
    print(models[m])
    for i in range(6):
        MAE, RMSE,PREC= Evaluation(test_label[:,i,:], test_predict_all[m][:,i,:])
        print('time:'+str(i+1)+' '+'MAE = '+str(MAE)+' '+'RMSE = '+str(RMSE)+' '+'PREC = '+str(PREC))

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')
time:1 MAE = 44.653916352473466 RMSE = 3664.1993894854418 PREC = 0.720678264241887
time:2 MAE = 55.09549764555813 RMSE = 3665.2503788432623 PREC = 0.6041842684801728
time:3 MAE = 62.96056849465238 RMSE = 3665.626613557556 PREC = 0.5303466807019488
time:4 MAE = 68.94776678981528 RMSE = 3665.6857981054795 PREC = 0.48189725619468415
time:5 MAE = 74.24632159457549 RMSE = 3666.0316757381524 PREC = 0.4411937838474217
time:6 MAE = 78.50419618280314 RMSE = 3666.975367270588 PREC = 0.41475297441792824
